In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import datetime
from urllib.request import Request, urlopen
import re
import numpy as np
import pandas_datareader.data as web
from fredapi import Fred
from scipy.interpolate import interp1d
from scipy import interpolate
import yahoo_fin.stock_info as si
import numpy_financial as npf 



In [2]:
# Inspección de WIKIPEDIA para crear lista de TICKERS

# Url de wiki donde esta la lista del s&p 500
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
data = requests.get(url).text

# analizar htlm data 
soup = BeautifulSoup(data, 'html.parser')

# Todas las tablas que tienen tickers estan en la clase "external text"
sopa = soup.findAll( class_='external text')

In [3]:
#Creo lista para almacenar tickers y borro lo q trajo que dice reports
tickers =[]
for n in range(0, len(sopa)):
    if sopa[n].text != 'reports':
        tickers.append(sopa[n].text)

#Creo lista con los tickers sin ningun otro objeto extra 
tickers_f = []
for r in range(0,len(tickers)):
    if tickers[r].count(' ') == 0 and  tickers[r].count('.com') == 0:
        tickers_f.append(tickers[r])
tickers_f    

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ATVI',
 'ADM',
 'ADBE',
 'ADP',
 'AAP',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BALL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'WRB',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BG',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CDAY',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA'

In [4]:
# Credenciales depende del USUARIO y comop se vaya actualizando el navegador. 
headers = {
    'User-agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
}

# URL a inspeccionar
url = "https://www.marketwatch.com/investing/stock/" + tickers_f[1]      + "/financials/income/quarter"
response = requests.get(url, headers=headers).text
soup = BeautifulSoup(response, "lxml")
company = tickers_f[1] 

# Fechas 

fechas = soup.find_all("thead", class_= "table__header")

fecha = fechas[0].text.split("\n")
fecha_lista =  [x for x in fecha if x != '']
fecha_lista = fecha_lista[2:7]
fecha_lista

# IS

date_2220 = soup.find_all("td", class_="overflow__cell")


for row in date_2220:
    cells = row.find_all("span")

data = []
for row in date_2220:
    cells = row.find_all("span")
    row_data = [cell.text for cell in cells]
    data.append(row_data)

merged_list = []
merged_list = [item for sublist in data for item in sublist]

merged_list



for i in range(0,len(merged_list)):
   
        
    merged_list[i] = str(merged_list[i]).replace('M', '')
    merged_list[i] = str(merged_list[i]).replace('(', '')
    merged_list[i] = str(merged_list[i]).replace(')', '')
    merged_list[i] = str(merged_list[i]).replace('-', '0')
    if "B" in merged_list[i]:
           merged_list[i] = round(float(str(merged_list[i]).replace('B', ''))*1000,3)

    elif  "%" in merged_list[i]:
        merged_list[i] = merged_list[i].replace("%","")
        merged_list[i] = merged_list[i].replace(",","")
        merged_list[i] = round(float(merged_list[i])/100,3)
        
    elif  "K" in merged_list[i]:
        merged_list[i] = merged_list[i].replace("K","")
        merged_list[i] = merged_list[i].replace(",","")
        merged_list[i] = round(float(merged_list[i])/100,3)
 
    
my_list = [x for x in merged_list if x]
sublists = [my_list[i:i+5] for i in range(0, len(my_list), 5)]   
    
items = soup.find_all("td", class_="overflow__cell fixed--column")

for n in range(0,len(items)):
    items[n] = items[n].text.split('\n')[2]
    
df_Income_Statement = pd.DataFrame(sublists, columns= fecha_lista, index= items)

Income_statement_company = df_Income_Statement.astype(float)
Income_statement_company
    
    

,31-Dec-2021,31-Mar-2022,30-Jun-2022,30-Sep-2022,31-Dec-2022
Sales/Revenue,995.50,977.700,965.900,874.200,936.100
Sales Growth,0.00,0.018,0.012,0.095,0.071
Cost of Goods Sold (COGS) incl. D&A,635.00,636.200,632.200,577.400,589.800
COGS Growth,0.00,0.002,0.006,0.087,0.021
COGS excluding D&A,615.10,615.900,614.200,559.200,569.400
Depreciation & Amortization Expense,19.90,20.300,18.000,18.200,20.400
Depreciation,0.00,0.000,0.000,0.000,0.000
Amortization of Intangibles,0.00,0.000,0.000,0.000,0.000
Gross Income,360.50,341.500,333.700,296.800,346.300
Gross Income Growth,0.00,0.053,0.023,0.111,0.167


In [5]:
# URL a inspeccionar
url = "https://www.marketwatch.com/investing/stock/" + tickers_f[1]      + "/financials/balance-sheet/quarter"
response = requests.get(url, headers=headers).text
soup = BeautifulSoup(response, "lxml")



# IS

date_2220 = soup.find_all("td", class_="overflow__cell")


for row in date_2220:
    cells = row.find_all("span")

data = []
for row in date_2220:
    cells = row.find_all("span")
    row_data = [cell.text for cell in cells]
    data.append(row_data)

merged_list = []
merged_list = [item for sublist in data for item in sublist]

merged_list



for i in range(0,len(merged_list)):
    
    merged_list[i] = str(merged_list[i]).replace('M', '')
    merged_list[i] = str(merged_list[i]).replace('(', '')
    merged_list[i] = str(merged_list[i]).replace(')', '')
    merged_list[i] = str(merged_list[i]).replace('-', '0')
    
    if "B" in merged_list[i]:
        merged_list[i] = round(float(str(merged_list[i]).replace('B', ''))*1000,3)
        
    
    elif "%" in merged_list[i]:
        merged_list[i] = merged_list[i].replace("%","")
        merged_list[i] = merged_list[i].replace(",","")
        merged_list[i] = round(float(merged_list[i])/100,3)
        
    elif  "K" in merged_list[i]:
        merged_list[i] = merged_list[i].replace("K","")
        merged_list[i] = merged_list[i].replace(",","")
        merged_list[i] = round(float(merged_list[i])/100,3)
 

my_list = [x for x in merged_list if x]
sublists = [my_list[i:i+5] for i in range(0, len(my_list), 5)]   
    
items = soup.find_all("td", class_="overflow__cell fixed--column")

for n in range(0,len(items)):
    items[n] = items[n].text.split('\n')[2]
    
df_BalanceSheet = pd.DataFrame(sublists, columns= fecha_lista, index= items)
Balance_Sheet_company = df_BalanceSheet.astype(float)
Balance_Sheet_company
    

,31-Dec-2021,31-Mar-2022,30-Jun-2022,30-Sep-2022,31-Dec-2022
Cash & Short Term Investments,631.400,579.400,459.400,417.100,481.800
Cash & Short Term Investments Growth,0.000,0.082,0.207,0.092,0.155
Cash Only,443.300,405.800,359.400,358.800,391.200
Short-Term Investments,0.000,0.000,0.000,0.000,0.000
Cash & ST Investments / Total Assets,0.182,0.169,0.138,0.129,0.145
...,...,...,...,...,...
Total Shareholders' Equity,1830.000,1810.000,1790.000,1750.000,1750.000
Total Shareholders' Equity / Total Assets,0.527,0.527,0.540,0.541,0.525
Accumulated Minority Interest,0.000,0.000,0.000,0.000,0.000
Total Equity,1830.000,1810.000,1790.000,1750.000,1750.000


In [6]:
revenue_g = (Income_statement_company.iloc[:,0]['Sales/Revenue'] - Income_statement_company.iloc[:,1]['Sales/Revenue'])/Income_statement_company.iloc[:,1]['Sales/Revenue']
revenue_g


0.018205993658586432

In [7]:
income_statement_1 = pd.DataFrame(columns =['current_year', 'as_%_of_revenue'], dtype = object)
income_statement_1['current_year'] = Income_statement_company.iloc[:,0]

income_statement_1['as_%_of_revenue'] = income_statement_1['current_year']/(income_statement_1.iloc[:,0]['Sales/Revenue'])

income_statement_1['next_year'] =  (income_statement_1['current_year']['Sales/Revenue'] * (1+revenue_g)) * income_statement_1['as_%_of_revenue'] 


income_statement_1




,current_year,as_%_of_revenue,next_year
Sales/Revenue,995.50,1.000000,1013.624067
Sales Growth,0.00,0.000000,0.000000
Cost of Goods Sold (COGS) incl. D&A,635.00,0.637870,646.560806
COGS Growth,0.00,0.000000,0.000000
COGS excluding D&A,615.10,0.617880,626.298507
Depreciation & Amortization Expense,19.90,0.019990,20.262299
Depreciation,0.00,0.000000,0.000000
Amortization of Intangibles,0.00,0.000000,0.000000
Gross Income,360.50,0.362130,367.063261
Gross Income Growth,0.00,0.000000,0.000000


In [8]:
#Pronosticar los próximos 5 años del estado de resultados
income_statement_1['next_year'] =  (income_statement_1['current_year']['Sales/Revenue'] * (1+revenue_g)) * income_statement_1['as_%_of_revenue'] 
income_statement_1['next_2_year'] =  (income_statement_1['next_year']['Sales/Revenue'] * (1+revenue_g)) * income_statement_1['as_%_of_revenue'] 
income_statement_1['next_3_year'] =  (income_statement_1['next_2_year']['Sales/Revenue'] * (1+revenue_g)) * income_statement_1['as_%_of_revenue'] 
income_statement_1['next_4_year'] =  (income_statement_1['next_3_year']['Sales/Revenue'] * (1+revenue_g)) * income_statement_1['as_%_of_revenue'] 
income_statement_1['next_5_year'] =  (income_statement_1['next_4_year']['Sales/Revenue'] * (1+revenue_g)) * income_statement_1['as_%_of_revenue'] 


income_statement_1 = income_statement_1.fillna(0)
income_statement_1


,current_year,as_%_of_revenue,next_year,next_2_year,next_3_year,next_4_year,next_5_year
Sales/Revenue,995.50,1.000000,1013.624067,1032.078100,1050.868107,1070.000205,1089.480622
Sales Growth,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Cost of Goods Sold (COGS) incl. D&A,635.00,0.637870,646.560806,658.332088,670.317678,682.521477,694.947459
COGS Growth,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
COGS excluding D&A,615.10,0.617880,626.298507,637.700893,649.310872,661.132221,673.168790
Depreciation & Amortization Expense,19.90,0.019990,20.262299,20.631195,21.006806,21.389256,21.778668
Depreciation,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Amortization of Intangibles,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Gross Income,360.50,0.362130,367.063261,373.746012,380.550430,387.478728,394.533164
Gross Income Growth,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:
balance_sheet_1 =  pd.DataFrame(columns =['current_year', 'as_%_of_revenue'], dtype = object)
balance_sheet_1['current_year'] = Balance_Sheet_company.iloc[:,0]
balance_sheet_1['as_%_of_revenue'] = balance_sheet_1['current_year'] / income_statement_1['current_year']['Sales/Revenue']


#Pronosticar los próximos 5 años del balance general

balance_sheet_1['next_year'] =  income_statement_1['next_year']['Sales/Revenue'] * balance_sheet_1['as_%_of_revenue']
balance_sheet_1['next_2_year'] =  income_statement_1['next_2_year']['Sales/Revenue'] * balance_sheet_1['as_%_of_revenue']
balance_sheet_1['next_3_year'] =  income_statement_1['next_3_year']['Sales/Revenue'] * balance_sheet_1['as_%_of_revenue'] 
balance_sheet_1['next_4_year'] =  income_statement_1['next_4_year']['Sales/Revenue']  * balance_sheet_1['as_%_of_revenue'] 
balance_sheet_1['next_5_year'] =  income_statement_1['next_5_year']['Sales/Revenue'] * balance_sheet_1['as_%_of_revenue']

balance_sheet_1

,current_year,as_%_of_revenue,next_year,next_2_year,next_3_year,next_4_year,next_5_year
Cash & Short Term Investments,631.400,0.634254,642.895264,654.599812,666.517452,678.652064,691.007599
Cash & Short Term Investments Growth,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Cash Only,443.300,0.445304,451.370717,459.588369,467.955632,476.475230,485.149935
Short-Term Investments,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Cash & ST Investments / Total Assets,0.182,0.000183,0.185313,0.188687,0.192123,0.195620,0.199182
...,...,...,...,...,...,...,...
Total Shareholders' Equity,1830.000,1.838272,1863.316968,1897.240505,1931.781654,1966.951658,2002.761968
Total Shareholders' Equity / Total Assets,0.527,0.000529,0.536595,0.546364,0.556311,0.566439,0.576752
Accumulated Minority Interest,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Total Equity,1830.000,1.838272,1863.316968,1897.240505,1931.781654,1966.951658,2002.761968


In [10]:
#Pronosticar los próximos 5 años FFC

CF_forecast = {}
CF_forecast['next_year'] = {}
CF_forecast['next_year']['netIncome'] = income_statement_1['next_year']['Net Income']
CF_forecast


CF_forecast['next_year']['inc_depreciation'] = income_statement_1['next_year']['Depreciation & Amortization Expense'] - income_statement_1['current_year']['Depreciation & Amortization Expense']
CF_forecast['next_year']['inc_receivables'] = balance_sheet_1['next_year']['Total Accounts Receivable'] - balance_sheet_1['current_year']['Total Accounts Receivable']
CF_forecast['next_year']['inc_inventory'] = balance_sheet_1['next_year']['Inventories'] - balance_sheet_1['current_year']['Inventories']
CF_forecast['next_year']['inc_payables'] = balance_sheet_1['next_year']['Accounts Payable'] - balance_sheet_1['current_year']['Accounts Payable']
CF_forecast['next_year']['CF_operations'] = CF_forecast['next_year']['netIncome'] + CF_forecast['next_year']['inc_depreciation'] + (CF_forecast['next_year']['inc_receivables'] * -1) + (CF_forecast['next_year']['inc_inventory'] *-1) + CF_forecast['next_year']['inc_payables']
CF_forecast['next_year']['CAPEX'] = balance_sheet_1['next_year']['Net Property, Plant & Equipment'] - balance_sheet_1['current_year']['Net Property, Plant & Equipment'] + income_statement_1['next_year']['Depreciation & Amortization Expense']

CF_forecast['next_year']['FCF'] = CF_forecast['next_year']['CAPEX'] + CF_forecast['next_year']['CF_operations']
CF_forecast


CF_forecast['next_2_year'] = {}
CF_forecast['next_2_year']['netIncome'] = income_statement_1['next_2_year']['Net Income']

CF_forecast['next_2_year']['inc_depreciation'] = income_statement_1['next_2_year']['Depreciation & Amortization Expense'] - income_statement_1['next_year']['Depreciation & Amortization Expense']
CF_forecast['next_2_year']['inc_receivables'] = balance_sheet_1['next_2_year']['Total Accounts Receivable'] - balance_sheet_1['next_year']['Total Accounts Receivable']
CF_forecast['next_2_year']['inc_inventory'] = balance_sheet_1['next_2_year']['Inventories'] - balance_sheet_1['next_year']['Inventories']
CF_forecast['next_2_year']['inc_payables'] = balance_sheet_1['next_2_year']['Accounts Payable'] - balance_sheet_1['next_year']['Accounts Payable']
CF_forecast['next_2_year']['CF_operations'] = CF_forecast['next_2_year']['netIncome'] + CF_forecast['next_2_year']['inc_depreciation'] + (CF_forecast['next_2_year']['inc_receivables'] * -1) + (CF_forecast['next_2_year']['inc_inventory'] *-1) + CF_forecast['next_2_year']['inc_payables']
CF_forecast['next_2_year']['CAPEX'] = balance_sheet_1['next_2_year']['Net Property, Plant & Equipment'] - balance_sheet_1['next_year']['Net Property, Plant & Equipment'] + income_statement_1['next_2_year']['Depreciation & Amortization Expense']
CF_forecast['next_2_year']['FCF'] = CF_forecast['next_2_year']['CAPEX'] + CF_forecast['next_2_year']['CF_operations']


CF_forecast

CF_forecast['next_3_year'] = {}
CF_forecast['next_3_year']['netIncome'] = income_statement_1['next_3_year']['Net Income']

CF_forecast['next_3_year']['inc_depreciation'] = income_statement_1['next_3_year']['Depreciation & Amortization Expense'] - income_statement_1['next_2_year']['Depreciation & Amortization Expense']
CF_forecast['next_3_year']['inc_receivables'] = balance_sheet_1['next_3_year']['Total Accounts Receivable'] - balance_sheet_1['next_2_year']['Total Accounts Receivable']
CF_forecast['next_3_year']['inc_inventory'] = balance_sheet_1['next_3_year']['Inventories'] - balance_sheet_1['next_2_year']['Inventories']
CF_forecast['next_3_year']['inc_payables'] = balance_sheet_1['next_3_year']['Accounts Payable'] - balance_sheet_1['next_2_year']['Accounts Payable']
CF_forecast['next_3_year']['CF_operations'] = CF_forecast['next_3_year']['netIncome'] + CF_forecast['next_3_year']['inc_depreciation'] + (CF_forecast['next_3_year']['inc_receivables'] * -1) + (CF_forecast['next_3_year']['inc_inventory'] *-1) + CF_forecast['next_3_year']['inc_payables']
CF_forecast['next_3_year']['CAPEX'] = balance_sheet_1['next_3_year']['Net Property, Plant & Equipment'] - balance_sheet_1['next_2_year']['Net Property, Plant & Equipment'] + income_statement_1['next_3_year']['Depreciation & Amortization Expense']
CF_forecast['next_3_year']['FCF'] = CF_forecast['next_3_year']['CAPEX'] + CF_forecast['next_3_year']['CF_operations']


CF_forecast

CF_forecast['next_4_year'] = {}
CF_forecast['next_4_year']['netIncome'] = income_statement_1['next_4_year']['Net Income']

CF_forecast['next_4_year']['inc_depreciation'] = income_statement_1['next_4_year']['Depreciation & Amortization Expense'] - income_statement_1['next_3_year']['Depreciation & Amortization Expense']
CF_forecast['next_4_year']['inc_receivables'] = balance_sheet_1['next_4_year']['Total Accounts Receivable'] - balance_sheet_1['next_3_year']['Total Accounts Receivable']
CF_forecast['next_4_year']['inc_inventory'] = balance_sheet_1['next_4_year']['Inventories'] - balance_sheet_1['next_3_year']['Inventories']
CF_forecast['next_4_year']['inc_payables'] = balance_sheet_1['next_4_year']['Accounts Payable'] - balance_sheet_1['next_3_year']['Accounts Payable']
CF_forecast['next_4_year']['CF_operations'] = CF_forecast['next_4_year']['netIncome'] + CF_forecast['next_4_year']['inc_depreciation'] + (CF_forecast['next_4_year']['inc_receivables'] * -1) + (CF_forecast['next_4_year']['inc_inventory'] *-1) + CF_forecast['next_4_year']['inc_payables']
CF_forecast['next_4_year']['CAPEX'] = balance_sheet_1['next_4_year']['Net Property, Plant & Equipment'] - balance_sheet_1['next_3_year']['Net Property, Plant & Equipment'] + income_statement_1['next_4_year']['Depreciation & Amortization Expense']
CF_forecast['next_4_year']['FCF'] = CF_forecast['next_4_year']['CAPEX'] + CF_forecast['next_4_year']['CF_operations']

CF_forecast


CF_forecast['next_5_year'] = {}
CF_forecast['next_5_year']['netIncome'] = income_statement_1['next_5_year']['Net Income']

CF_forecast['next_5_year']['inc_depreciation'] = income_statement_1['next_5_year']['Depreciation & Amortization Expense'] - income_statement_1['next_4_year']['Depreciation & Amortization Expense']
CF_forecast['next_5_year']['inc_receivables'] = balance_sheet_1['next_5_year']['Total Accounts Receivable'] - balance_sheet_1['next_4_year']['Total Accounts Receivable']
CF_forecast['next_5_year']['inc_inventory'] = balance_sheet_1['next_5_year']['Inventories'] - balance_sheet_1['next_4_year']['Inventories']
CF_forecast['next_5_year']['inc_payables'] = balance_sheet_1['next_5_year']['Accounts Payable'] - balance_sheet_1['next_4_year']['Accounts Payable']
CF_forecast['next_5_year']['CF_operations'] = CF_forecast['next_5_year']['netIncome'] + CF_forecast['next_5_year']['inc_depreciation'] + (CF_forecast['next_5_year']['inc_receivables'] * -1) + (CF_forecast['next_5_year']['inc_inventory'] *-1) + CF_forecast['next_5_year']['inc_payables']
CF_forecast['next_5_year']['CAPEX'] = balance_sheet_1['next_5_year']['Net Property, Plant & Equipment'] - balance_sheet_1['next_4_year']['Net Property, Plant & Equipment'] + income_statement_1['next_5_year']['Depreciation & Amortization Expense']
CF_forecast['next_5_year']['FCF'] = CF_forecast['next_5_year']['CAPEX'] + CF_forecast['next_5_year']['CF_operations']

CF_forecast




{'next_year': {'netIncome': 142.14155671473864,
  'inc_depreciation': 0.362299273805867,
  'inc_receivables': 11.54988237700718,
  'inc_inventory': 8.150823360949119,
  'inc_payables': 13.57985066993956,
  'CF_operations': 136.38300092052776,
  'CAPEX': 31.899570420374175,
  'FCF': 168.28257134090194},
 'next_2_year': {'netIncome': 144.72938499490877,
  'inc_depreciation': 0.36889529208728433,
  'inc_receivables': 11.760159462320303,
  'inc_inventory': 8.29921719937073,
  'inc_payables': 13.827085345120963,
  'CF_operations': 138.86598897042597,
  'CAPEX': 32.48033379715916,
  'FCF': 171.34632276758515},
 'next_3_year': {'netIncome': 147.36432726033718,
  'inc_depreciation': 0.3756113974357156,
  'inc_receivables': 11.974264850915347,
  'inc_inventory': 8.45031269507382,
  'inc_payables': 14.078821173231063,
  'CF_operations': 141.39418228501478,
  'CAPEX': 33.07167054829909,
  'FCF': 174.46585283331387},
 'next_4_year': {'netIncome': 150.0472412679407,
  'inc_depreciation': 0.38244977

In [11]:
#Llevar los flujos de caja proyectados a un DataFrame de pandas
CF_forec = pd.DataFrame.from_dict(CF_forecast,orient='columns')
CF_forec

#Formatear el DataFrame con separadores de miles
pd.options.display.float_format = '{:,.0f}'.format


In [12]:
# Income_statement_company.iloc



def interest_coveraga_and_RF(company):

#     EBIT= income_statement.iloc[:,0]['NetIncome'] - income_statement.iloc[:,0]['ProvisionforIncomeTaxes'] - income_statement.iloc[:,0]['InterestExpense(Income)NetOperating']
    EBIT = Income_statement_company.iloc[:,0]['EBITDA'] - Income_statement_company.iloc[:,0]['Depreciation & Amortization Expense'] 


    interest_expense =  Income_statement_company.iloc[:,0]['Interest Expense']

    try:

        interest_coverage_ratio = EBIT / interest_expense


    except:

        interest_coverage_ratio = EBIT


    #RF
    start = datetime.datetime(2019, 7, 10)

    end= datetime.datetime.today().strftime('%Y-%m-%d')
    #end = datetime.datetime(2020, 7, 10)

    Treasury = web.DataReader(['TB1YR'], 'fred', start, end)
    RF = float(Treasury.iloc[-1])
    RF = RF/100

    return [RF,interest_coverage_ratio]



In [24]:
def cost_of_debt(company, RF,interest_coverage_ratio):

    # Lista nombres bonos

    nom_bonds = ['BAMLC0A1CAAA', 'BAMLC0A2CAA', 'BAMLC0A3CA', 'BAMLC0A4CBBB', 'BAMLH0A1HYBB', 'BAMLH0A2HYB', 'BAMLH0A3HYC']

    spreads = []
    for s in nom_bonds:
        fred = Fred( api_key = '8ce7f9c8fce20e2e5d0cb6b113cf79bf')
        data = fred.get_series(s)
        time.sleep(10)
        data = data[len(data)-1]
        spreads.append(data)

    tasa = spreads
    total_spreads = []

    X = [1, 2, 4, 6, 8, 10, 12]
    Value_interpolate = [ 3, 5, 7, 9, 11]

    for interp in Value_interpolate:

        y_interp = interp1d(X, tasa)
        spread =  round(float(y_interp(interp)),2)
        total_spreads.append(spread)

    for t in range(0, len(Value_interpolate)):
        tasa.insert(Value_interpolate[t]-1, total_spreads[t])


    xi = [1, 2, 3,4,5,6,7,8,9,10,11, 12]
    yi = tasa
    v_inter = [13,14,15]
    r = []

    for t in v_inter:
        f = interpolate.interp1d(xi, yi, fill_value = "extrapolate")
        valor = round(float(f(t)),2)
        r.append(valor)


    if interest_coverage_ratio > 8.5:
    #Rating is AAA
        credit_spread = tasa[0]/100
    if (interest_coverage_ratio > 6.5) & (interest_coverage_ratio <= 8.5):
    #Rating is AA
        credit_spread = tasa[1]/100
    if (interest_coverage_ratio > 5.5) & (interest_coverage_ratio <=  6.5):
    #Rating is A+
        credit_spread = tasa[2]/100
    if (interest_coverage_ratio > 4.25) & (interest_coverage_ratio <=  5.49):
    #Rating is A
        credit_spread = tasa[3]/100
    if (interest_coverage_ratio > 3) & (interest_coverage_ratio <=  4.25):
    #Rating is A-
        credit_spread = tasa[4]/100
    if (interest_coverage_ratio > 2.5) & (interest_coverage_ratio <=  3):
    #Rating is BBB
        credit_spread = tasa[5]/100
    if (interest_coverage_ratio > 2.25) & (interest_coverage_ratio <=  2.5):
    #Rating is BB+
        credit_spread = tasa[6]/100
    if (interest_coverage_ratio > 2) & (interest_coverage_ratio <=  2.25):
    #Rating is BB
        credit_spread = tasa[7]/100
    if (interest_coverage_ratio > 1.75) & (interest_coverage_ratio <=  2):
    #Rating is B+
        credit_spread = tasa[8]/100
    if (interest_coverage_ratio > 1.5) & (interest_coverage_ratio <=  1.75):
    #Rating is B
        credit_spread = tasa[9]/100
    if (interest_coverage_ratio > 1.25) & (interest_coverage_ratio <=  1.5):
    #Rating is B-
        credit_spread = tasa[10]/100
    if (interest_coverage_ratio > 0.8) & (interest_coverage_ratio <=  1.25):
    #Rating is CCC
        credit_spread = tasa[11]/100
    if (interest_coverage_ratio > 0.65) & (interest_coverage_ratio <=  0.8):
    #Rating is CC
        credit_spread = r[0]/100
    if (interest_coverage_ratio > 0.2) & (interest_coverage_ratio <=  0.65):
    #Rating is C
        credit_spread = r[1]/100
    if interest_coverage_ratio <=  0.2:
    #Rating is D
        credit_spread = r[2]/100

    cost_of_debt = RF + credit_spread

    return cost_of_debt



def costofequity(company):

    #RF o tasa libre de riesgo
    start = datetime.datetime(2019, 7, 10)
    end= datetime.datetime.today().strftime('%Y-%m-%d')
    #end = datetime.datetime(2020, 7, 10)

    Treasury = web.DataReader(['TB1YR'], 'fred', start, end)
    RF = float(Treasury.iloc[-1])
    RF = RF/100

    #Beta

    beta = si.get_quote_table(company)['Beta (5Y Monthly)']



    #Rentabilidad del mercado
    start = datetime.datetime(2010, 1, 1)
    end= datetime.datetime.today().strftime('%Y-%m-%d')

    SP500 = web.DataReader(['sp500'], 'fred', start, end)
      # Quitar todos los valores no numéricos con el método drop.
    SP500.dropna(inplace = True)

    SP500yearlyreturn = (SP500['sp500'].iloc[-1]/ SP500['sp500'].iloc[-267])-1
    
    
#     dates = [date(2013,2,4), date(end)]
#     amounts = [-SP500['sp500'].iloc[-2608], SP500['sp500'].iloc[-1]]

#     # feed columnar data
#     xirr(dates, amounts)

#     # feed tuples
#     xirr(zip(dates, amounts))

#     # feed DataFrame
#     import pandas as pd
#     xirr(pd.DataFrame({"dates": dates, "amounts": amounts}))
    
    

    cost_of_equity = RF+(beta*(SP500yearlyreturn - RF))

    return cost_of_equity







def wacc(company):
    ETR = Income_statement_company.iloc[:,0]["Income Tax"]/Income_statement_company.iloc[:,0]["Pretax Income"]


    Lterm_debt = Balance_Sheet_company.iloc[:,0]['Long-Term Debt']    
    Sterm_debt = Balance_Sheet_company.iloc[:,1]['ST Debt & Current Portion LT Debt']


    Debt_to = (Sterm_debt +  Lterm_debt ) / ((Sterm_debt +  Lterm_debt ) + Balance_Sheet_company.iloc[:,0]["Total Equity"]) 
    equity_to =  Balance_Sheet_company.iloc[:,0]["Total Equity"] / ( (Sterm_debt +  Lterm_debt ) + Balance_Sheet_company.iloc[:,0]["Total Equity"])






    WACC = (kd*(1-ETR)*Debt_to) + (ke*equity_to)

    return WACC

In [25]:
RF_and_IntCov = interest_coveraga_and_RF(company)
RF = RF_and_IntCov[0]
interest_coverage_ratio = RF_and_IntCov[1]
ke = costofequity(company)
kd = cost_of_debt(company,RF,interest_coverage_ratio)
wacc_company = wacc(company)

C:\Users\jsaenz\Anaconda3\lib\site-packages\yahoo_fin\stock_info.py:295: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = tables[0].append(tables[1])
C:\Users\jsaenz\Anaconda3\lib\site-packages\yahoo_fin\stock_info.py:302: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(quote_price)


In [26]:
RF_and_IntCov, RF, interest_coverage_ratio, ke, kd, wacc_company

([0.047, 125.92857142857143],
 0.047,
 125.92857142857143,
 -0.13798043989112152,
 0.0533,
 -0.11802067629243373)

In [37]:
FCF_List = CF_forec.iloc[-1].values.tolist()

npv = npf.npv(wacc_company,FCF_List)

#Valor terminal
LTGrowth = 0.02

Terminal_value = (CF_forecast['next_5_year']['FCF'] * (1+ LTGrowth)) /(wacc_company  - LTGrowth)

Terminal_value_Discounted = Terminal_value/(1+wacc_company)**4
Terminal_value_Discounted

target_equity_value = Terminal_value_Discounted + npv
debt = balance_sheet_1.iloc[:,0]['Long-Term Debt']  + balance_sheet_1.iloc[:,0]['Short Term Debt']
target_value = target_equity_value - debt


numbre_of_shares = (Balance_Sheet_company.iloc[:,0]['Common Equity (Total)']- \
                    Balance_Sheet_company.iloc[:,0]['Preferred Stock (Carrying Value)']-\
                    Balance_Sheet_company.iloc[:,0]['ESOP Debt Guarantee'])/\
                    (Balance_Sheet_company.iloc[:,0]['Common Stock Par/Carry Value'])


# numbre_of_shares = requests.get(f'https://financialmodelingprep.com/api/v3/enterprise-values/{company}?apikey={demo}').json()
# numbre_of_shares = numbre_of_shares[0]['numberOfShares']

#  numbre_of_shares = balance_sheet.iloc[:,0]['Common Equity (Total)']


# if balance_sheet.iloc[:,0]['TotalCommonSharesOutstanding'] == 0:

#     numbre_of_shares = balance_sheet.iloc[:,0]['Common Equity (Total)']
    
# else:
    
#     numbre_of_shares = balance_sheet.iloc[:,0]['TotalCommonSharesOutstanding']


target_price_per_share = target_value/numbre_of_shares
target_price_per_share

print(company + ' forecasted price per stock is ' + str(target_price_per_share) )
print('the forecast is based on the following assumptions: '+ 'revenue growth: ' + str(revenue_g) + ' Cost of Capital: ' + str(wacc_company) )
print('perpetuity growth: ' + str(LTGrowth)  )
print(url)

AOS forecasted price per stock is -207.81200830634398
the forecast is based on the following assumptions: revenue growth: 0.018205993658586432 Cost of Capital: -0.11802067629243373
perpetuity growth: 0.02
https://www.marketwatch.com/investing/stock/AOS/financials/balance-sheet/quarter


In [ ]:
RF_and_IntCov = interest_coveraga_and_RF(tickers_f[1])
RF = RF_and_IntCov[0]
interest_coverage_ratio = RF_and_IntCov[1]
ke = costofequity(tickers_f[1])
kd = cost_of_debt(tickers_f[1],RF,interest_coverage_ratio)
wacc_company = wacc(tickers_f[1])

RF_and_IntCov, RF, interest_coverage_ratio, ke, kd, wacc_company

In [ ]:
RF_and_IntCov, RF, interest_coverage_ratio, ke, kd, wacc_company

In [ ]:
url

In [ ]:
nom_bonds

In [34]:

numbre_of_shares

6.199186991869919

In [ ]:
nom_bonds = ['BAMLC0A1CAAA', 'BAMLC0A2CAA', 'BAMLC0A3CA', 'BAMLC0A4CBBB', 'BAMLH0A1HYBB', 'BAMLH0A2HYB', 'BAMLH0A3HYC']

spreads = []
for s in nom_bonds:
    fred = Fred( api_key = '8ce7f9c8fce20e2e5d0cb6b113cf79bf')
    data = fred.get_series(s)
    data = data[len(data)-1]
    spreads.append(data)

tasa = spreads
total_spreads = []

X = [1, 2, 4, 6, 8, 10, 12]
Value_interpolate = [ 3, 5, 7, 9, 11]

for interp in Value_interpolate:

    y_interp = interp1d(X, tasa)
    spread =  round(float(y_interp(interp)),2)
    total_spreads.append(spread)

for t in range(0, len(Value_interpolate)):
    tasa.insert(Value_interpolate[t]-1, total_spreads[t])


xi = [1, 2, 3,4,5,6,7,8,9,10,11, 12]
yi = tasa
v_inter = [13,14,15]
r = []

for t in v_inter:
    f = interpolate.interp1d(xi, yi, fill_value = "extrapolate")
    valor = round(float(f(t)),2)
    r.append(valor)


if interest_coverage_ratio > 8.5:
#Rating is AAA
    credit_spread = tasa[0]/100
if (interest_coverage_ratio > 6.5) & (interest_coverage_ratio <= 8.5):
#Rating is AA
    credit_spread = tasa[1]/100
if (interest_coverage_ratio > 5.5) & (interest_coverage_ratio <=  6.5):
#Rating is A+
    credit_spread = tasa[2]/100
if (interest_coverage_ratio > 4.25) & (interest_coverage_ratio <=  5.49):
#Rating is A
    credit_spread = tasa[3]/100
if (interest_coverage_ratio > 3) & (interest_coverage_ratio <=  4.25):
#Rating is A-
    credit_spread = tasa[4]/100
if (interest_coverage_ratio > 2.5) & (interest_coverage_ratio <=  3):
#Rating is BBB
    credit_spread = tasa[5]/100
if (interest_coverage_ratio > 2.25) & (interest_coverage_ratio <=  2.5):
#Rating is BB+
    credit_spread = tasa[6]/100
if (interest_coverage_ratio > 2) & (interest_coverage_ratio <=  2.25):
#Rating is BB
    credit_spread = tasa[7]/100
if (interest_coverage_ratio > 1.75) & (interest_coverage_ratio <=  2):
#Rating is B+
    credit_spread = tasa[8]/100
if (interest_coverage_ratio > 1.5) & (interest_coverage_ratio <=  1.75):
#Rating is B
    credit_spread = tasa[9]/100
if (interest_coverage_ratio > 1.25) & (interest_coverage_ratio <=  1.5):
#Rating is B-
    credit_spread = tasa[10]/100
if (interest_coverage_ratio > 0.8) & (interest_coverage_ratio <=  1.25):
#Rating is CCC
    credit_spread = tasa[11]/100
if (interest_coverage_ratio > 0.65) & (interest_coverage_ratio <=  0.8):
#Rating is CC
    credit_spread = r[0]/100
if (interest_coverage_ratio > 0.2) & (interest_coverage_ratio <=  0.65):
#Rating is C
    credit_spread = r[1]/100
if interest_coverage_ratio <=  0.2:
#Rating is D
    credit_spread = r[2]/100

cost_of_debt = RF + credit_spread
cost_of_debt

In [ ]:
Non-Equity Reserves

Total Liabilities / Total Assets

Preferred Stock (Carrying Value)

Redeemable Preferred Stock

Non-Redeemable Preferred Stock

Common Equity (Total)

Common Equity / Total Assets

Common Stock Par/Carry Value

Retained Earnings

ESOP Debt Guarantee

Cumulative Translation Adjustment/Unrealized For. Exch. Gain

Unrealized Gain/Loss Marketable Securities

Revaluation Reserves

Treasury Stock

Total Shareholders' Equity

Total Shareholders' Equity / Total Assets

Accumulated Minority Interest

Total Equity

Liabilities & Shareholders' Equity